<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Библиотеки" data-toc-modified-id="Библиотеки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Библиотеки</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Финальное-тестирование" data-toc-modified-id="Финальное-тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Финальное тестирование</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок. Значение метрики качества *F1* должно быть не меньше 0.75. 

Постройте модель со значением 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Библиотеки

In [1]:
# стандартные библиотеки
import warnings
import os

# сторонние библиотеки
import numpy as np
import pandas as pd
import torch
import transformers
import nltk

# модули
from tqdm import notebook
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.utils import shuffle

from sklearn.model_selection import (
    GridSearchCV,
    cross_val_score,
    cross_val_predict,
    train_test_split
)
from sklearn.metrics import f1_score 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier

# настройки
warnings.filterwarnings("ignore")
nltk.download('stopwords')

# константы
RANDOM_SATATE = 0

PATH_YANDEX = 'datasets/toxic_comments.csv'    
PATH = 'C:/Users/igors/Downloads/toxic_comments.csv'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\igors\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Подготовка

Загрузим данные и осмотрим первые строки.

In [2]:
if os.path.exists(PATH):
    data = pd.read_csv(PATH, index_col=[0])
    display(data.head(5))
elif os.path.exists(PATH_YANDEX):
    data = pd.read_csv(PATH_YANDEX, index_col=[0])
    display(data.head(5))
else:
    print('Ошибка в считывании данных') 

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Проверим данные на нализичие пропусков.

In [3]:
data.isna().sum()

text     0
toxic    0
dtype: int64

Пропусков нет.

Т.к. будем работать с BERT, уменьшим выборку до 1000 объектов, сохранив баланс классов.

In [4]:
data[data['toxic'] == 0].count()

text     143106
toxic    143106
dtype: int64

In [5]:
data[data['toxic'] == 1].count()

text     16186
toxic    16186
dtype: int64

In [6]:
zeros = data[data['toxic'] == 0].sample(round(143/(143+16)*1000), random_state=RANDOM_SATATE).copy()

In [7]:
ones = data[data['toxic'] == 1].sample(round(16/(143+16)*1000), random_state=RANDOM_SATATE).copy()

In [8]:
stratified_sample = shuffle(pd.concat([zeros, ones]), random_state=RANDOM_SATATE)

Задействуем модель toxic-bert для токенизации и создания эмбеддингов, затем разобьём данные на обучающую и тестовую выборки.

In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained("unitary/toxic-bert")
model = transformers.BertModel.from_pretrained("unitary/toxic-bert")

tokenized = stratified_sample['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

features = np.concatenate(embeddings)
target = stratified_sample[:features.shape[0]]['toxic']

%store features
%store target

  0%|          | 0/10 [00:00<?, ?it/s]

Stored 'features' (ndarray)
Stored 'target' (Series)


Сохраняем значения features и target на случай нестабильной работы ядра.

### Вывод

В ходе потоготовки
- из данных была взята выборка из 1000 объектов с сохранением пропорций классов;
- тексты были токенизированы, после чего были подготовлены эмбеддинги с помощью модели toxic-bert;
- были сформированы признаки для дальнейшей классификации.

## Обучение

In [11]:
%store -r features
%store -r target

Разобьём признаки и целевые признаки на тренировочную и тестовую выборки.

In [12]:
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.25, random_state=RANDOM_SATATE)

Для решения задачи будем использовать модели LinerRegression, DecisionTreeClassifier, KNeighborsClassifier и RandomForestClassifier.
Подберём оптимальные параметры одновременно с оптимальным порогом классификации для каждой модели.

In [13]:
best_params = {'class_weight': None, 'penalty': None, 'C': None, 'threshold': None, 'F1': None}
best_score = 0

for class_weight in [None, 'balanced']:    
    for penalty in notebook.tqdm(['l1', 'l2', 'elasticnet', None]):
        for C in np.arange(0, 1.1, 0.2):
            for threshold in np.arange(0.1, 1.05, 0.1):
                try:
                    model = LogisticRegression(C=C, penalty=penalty, random_state=RANDOM_SATATE, 
                                               n_jobs=-1, class_weight=class_weight)
                    predict_proba = cross_val_predict(model, train_features, train_target, method='predict_proba')
                    predict = [1 if x[1] >= threshold else 0 for x in predict_proba]
                    score = f1_score(predict, train_target)
                    if score > best_score:
                        best_score = score
                        best_params['class_weight'] = class_weight
                        best_params['penalty'] = penalty
                        best_params['C'] = C
                        best_params['threshold'] = threshold
                        best_params['F1'] =  best_score                           
                except:
                    pass
                    
display(best_params)    

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'class_weight': None,
 'penalty': 'l2',
 'C': 0.2,
 'threshold': 0.7000000000000001,
 'F1': 0.951048951048951}

In [14]:
best_params_DT = {'class_weight': None, 'max_depth': None, 'threshold': None, 'F1': None}
best_score = 0
for class_weight in [None, 'balanced']:
    for max_depth in notebook.tqdm(range(1, 11)):
        for threshold in np.arange(0.1, 1.05, 0.1):
            model = DecisionTreeClassifier(max_depth=max_depth, random_state=RANDOM_SATATE, class_weight=class_weight)
            predict_proba = cross_val_predict(model, train_features, train_target, method='predict_proba')
            predict = [1 if x[1] >= threshold else 0 for x in predict_proba]
            score = f1_score(predict, train_target)
            if score > best_score:
                best_score = score
                best_params_DT['class_weight'] = class_weight
                best_params_DT['max_depth'] = max_depth
                best_params_DT['threshold'] = threshold
                best_params_DT['F1'] = best_score                           

display(best_params_DT)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'class_weight': 'balanced',
 'max_depth': 2,
 'threshold': 0.1,
 'F1': 0.9395973154362416}

In [15]:
best_params_KN = {'n_neighbors': None, 'threshold': None, 'F1': None}
best_score = 0
for n_neighbors in notebook.tqdm(range(1, 11)):
    for threshold in np.arange(0.1, 1.05, 0.1):
        model = KNeighborsClassifier(n_neighbors=n_neighbors)
        predict_proba = cross_val_predict(model, train_features, train_target, method='predict_proba')
        predict = [1 if x[1] >= threshold else 0 for x in predict_proba]
        score = f1_score(predict, train_target)
        if score > best_score:
            best_score = score 
            best_params_KN['n_neighbors'] = n_neighbors
            best_params_KN['threshold'] = threshold
            best_params_KN['F1'] = best_score
            
display(best_params_KN)

  0%|          | 0/10 [00:00<?, ?it/s]

{'n_neighbors': 3, 'threshold': 0.4, 'F1': 0.9583333333333333}

In [16]:
best_params_RF = {'class_weight': None, 'n_estimators': None, 'max_depth': None, 'threshold': None, 'F1': None}
best_score = 0
for class_weight in [None, 'balanced']:    
    for n_estimators in notebook.tqdm(range(10, 210, 50)):
        for max_depth in range(1, 11):
            for threshold in np.arange(0.1, 1.05, 0.1):
                model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=RANDOM_SATATE, class_weight=class_weight)
                predict_proba = cross_val_predict(model, train_features, train_target, method='predict_proba')
                predict = [1 if x[1] >= threshold else 0 for x in predict_proba]
                score = f1_score(predict, train_target)
                if score > best_score:
                    best_score = score
                    best_params_RF['class_weight'] = class_weight
                    best_params_RF['n_estimators'] = n_estimators
                    best_params_RF['max_depth'] = max_depth
                    best_params_RF['threshold'] = threshold
                    best_params_RF['F1'] = best_score                           

display(best_params_RF)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'class_weight': 'balanced',
 'n_estimators': 10,
 'max_depth': 2,
 'threshold': 0.6,
 'F1': 0.9589041095890412}

Все модели показали отличные результаты, тем не менее самое высокое значение F1-меры у RandomForestClassifier.

### Вывод

В ходе обучения 
- были подобраны оптимальные гиперпараметры и оптимальные пороги классификации для каждой модели
- было принято решение использовать в дальнейшем модель **RandomForestClassifier с гиперпараметрами `n_estimators` = 10, `max_depth` = 2, `class_weight` = 'balanced' и порогом классификации 0.6**, т.к. она показывает самые высокие значения F1-меры среди всех рассмотренных моделей.

## Финальное тестирование

Напишем класс для предсказаний со сдвигом порога классификации.

In [17]:
class ShiftedThresholdModel():
    
    
    def __init__(self, model, threshold):
        self.model = model
        self.threshold = threshold
    
    def predict(self, X):
        predict_proba = pd.Series([x[0] for x in self.model.predict_proba(X)])
        return predict_proba.apply(lambda x: 0 if x >= self.threshold else 1)

Рассчитаем значение F1-меры на тестовой выбоорке для рекомендуемой модели.

In [18]:
pre_model = RandomForestClassifier(n_estimators=10, max_depth=2, class_weight='balanced', 
                               random_state=RANDOM_SATATE, n_jobs=-1
                               ).fit(train_features, train_target)
threshold = 0.6
final_model = ShiftedThresholdModel(pre_model, threshold)
f1_score(test_target, final_model.predict(test_features))

0.9473684210526316

### Выводы

В результате финального тестирования значение F1-метрики рекомендуемой модели равно 0.95, что удовлетворяет условиям поставленной задачи.

## Выводы

В ходе потоготовки
- из данных была взята выборка из 1000 объектов с сохранением пропорций классов;
- тексты были токенизированы, после чего были подготовлены эмбеддинги с помощью модели DistilBert;
- были сформированы признаки для дальнейшей классификации.

В ходе обучения 
- были подобраны оптимальные гиперпараметры и оптимальные пороги классификации для каждой модели
- было принято решение использовать в дальнейшем модель **RandomForestClassifier с гиперпараметрами `n_estimators` = 10, `max_depth` = 2, `class_weight` = 'balanced' и порогом классификации 0.6**, т.к. она показывает самые высокие значения F1-меры среди всех рассмотренных моделей.

В результате финального тестирования значение F1-меры рекомендуемой модели равно 0.95, что удовлетворяет условиям поставленной задачи.